En el archivo auto.csv se encuentran los siguientes datos de diferentes automóviles:
    Cilindros
    Cilindrada
    Potencia
    Peso
    Aceleración
    Año del coche
    Origen
    Consumo (mpg)
Las unidades de las características de los automóviles no se encuentran en el sistema internacional. La variable “origen” es un código que identifica al país de origen.

Crea un modelo con él para que se pueda estimar el consumo de un vehículo a partir del resto de las variables.

In [77]:
# Preparamos el entorno
%pylab
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# importamos las librerias
import pandas as pd  # Python Data Analysis Library

# Leemos el fichero
autos_df = pd.DataFrame.from_csv('auto.csv', sep=',', index_col = None, header = 0)
autos_df_ori = autos_df
print(autos_df.describe())
print(autos_df.head())

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
        cylinders  displacement  horsepower       weight  acceleration  \
count  392.000000    392.000000  392.000000   392.000000    392.000000   
mean     5.471939    194.411990  104.469388  2977.584184     15.541327   
std      1.705783    104.644004   38.491160   849.402560      2.758864   
min      3.000000     68.000000   46.000000  1613.000000      8.000000   
25%      4.000000    105.000000   75.000000  2225.250000     13.775000   
50%      4.000000    151.000000   93.500000  2803.500000     15.500000   
75%      8.000000    275.750000  126.000000  3614.750000     17.025000   
max      8.000000    455.000000  230.000000  5140.000000     24.800000   

       model_year      origin         mpg  
count  392.000000  392.000000  392.000000  
mean    75.979592    1.576531   23.445918  
std      3.683737    0.805518    7.805007  
min     70.000000    1.000000    9.000000  
25%     73.000000

In [46]:
# Obtencion de los Componentes Principales (PCA)
from sklearn.decomposition import PCA

# Lo obtenemos para todas las columnas de datos del fichero que tengan sentido:
# Eliminamos la variable año ya que no se dispondra de ella en prevision: 
autos_df.pop('model_year')
# Eliminamos tambien la variable dependiente:
autos_df_y = autos_df.pop('mpg')
print(autos_df.head())
print(autos_df_y.head())
pca = PCA(n_components = len(autos_df.columns))
x_pca = pca.fit_transform(autos_df)
ix = 0
for columna in autos_df.columns:
    print('Varianza explicada por la compontente', autos_df.columns[ix], pca.explained_variance_ratio_[ix])
    ix = ix + 1

   cylinders  displacement  horsepower  weight  acceleration  origin
0          8         307.0       130.0  3504.0          12.0       1
1          8         350.0       165.0  3693.0          11.5       1
2          8         318.0       150.0  3436.0          11.0       1
3          8         304.0       150.0  3433.0          12.0       1
4          8         302.0       140.0  3449.0          10.5       1
0    18.0
1    15.0
2    18.0
3    16.0
4    17.0
Name: mpg, dtype: float64
Varianza explicada por la compontente cylinders 0.9975781679137055
Varianza explicada por la compontente displacement 0.0020616704067824973
Varianza explicada por la compontente horsepower 0.0003553288067194986
Varianza explicada por la compontente weight 3.95644505650346e-06
Varianza explicada por la compontente acceleration 5.164006698398552e-07
Varianza explicada por la compontente origin 3.600270663983065e-07


La mayor parte de la varianza viene explicada por la variable cilindros
### Procedemos a generar el modelo de regresion lineal

In [47]:
# Eliminamos caracteristicas que sean comb. lineal del resto mediante VIF
from sklearn.linear_model import LinearRegression

def calculateVIF(data):
    features = list(data.columns)
    num_features = len(features)
    
    model = LinearRegression()
    
    result = pd.DataFrame(index = ['VIF'], columns = features)
    result = result.fillna(0)
    
    for ite in range(num_features):
        x_features = features[:]
        y_featue = features[ite]
        x_features.remove(y_featue)
        
        x = data[x_features]
        y = data[y_featue]
        
        model.fit(data[x_features], data[y_featue])
        
        result[y_featue] = 1/(1 - model.score(data[x_features], data[y_featue]))
    
    return result

def selectDataUsingVIF(data, max_VIF = 5):
    result = data.copy(deep = True)
    
    VIF = calculateVIF(result)
    
    while VIF.as_matrix().max() > max_VIF:
        col_max = np.where(VIF == VIF.as_matrix().max())[1][0]
        features = list(result.columns)
        features.remove(features[col_max])
        result = result[features]
        
        VIF = calculateVIF(result)
        
    return result

calculateVIF(autos_df)

,cylinders,displacement,horsepower,weight,acceleration,origin
VIF,10.735047,21.739569,9.45148,10.526431,2.609676,1.772212


In [48]:
model_vars = selectDataUsingVIF(autos_df)
calculateVIF(model_vars)

,cylinders,acceleration,origin
VIF,1.915138,1.356643,1.495008


In [79]:
# Dividimos los datos en conjunto de test y entrenamiento
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(model_vars, autos_df_y)

print(X_train.describe())
print(X_test.describe())
print(y_train.describe())
print(y_test.describe())

        cylinders  acceleration      origin
count  294.000000    294.000000  294.000000
mean     5.394558     15.568707    1.612245
std      1.687367      2.814766    0.813277
min      3.000000      8.000000    1.000000
25%      4.000000     13.925000    1.000000
50%      4.000000     15.450000    1.000000
75%      6.000000     17.200000    2.000000
max      8.000000     24.800000    3.000000
       cylinders  acceleration     origin
count  98.000000     98.000000  98.000000
mean    5.704082     15.459184   1.469388
std     1.748102      2.596065   0.775981
min     4.000000      9.500000   1.000000
25%     4.000000     13.500000   1.000000
50%     6.000000     15.500000   1.000000
75%     8.000000     17.000000   2.000000
max     8.000000     21.000000   3.000000
count    294.000000
mean      23.827211
std        7.779364
min       10.000000
25%       17.525000
50%       23.350000
75%       29.375000
max       46.600000
Name: mpg, dtype: float64
count    98.000000
mean     22.302041
st

In [73]:
# Generamos el modelo y lo evaluamos
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error

modeloLineal = LinearRegression()
modeloLineal.fit(X_train, y_train)
prediccion = modeloLineal.predict(X_train)

print('R^2', modeloLineal.score(X_train, y_train))
print('Error cuadrático medio', mean_squared_error(prediccion, y_train))
print('Error absoluto medio', mean_absolute_error(prediccion, y_train))
print('Mediana del error absoluto', median_absolute_error(prediccion, y_train))
print('-----------------------------------------------------------------------------')
prediccion_test = modeloLineal.predict(X_test)
for index in range(len(prediccion_test)):
    print('prediccion:', prediccion_test[index], '; Real:', y_test.iloc[index])


R^2 0.6310097938101107
Error cuadrático medio 21.7449651451369
Error absoluto medio 3.4385753531775003
Mediana del error absoluto 2.470681786766125
-----------------------------------------------------------------------------
prediccion: 28.057476338034533 ; Real: 30.5
prediccion: 14.710606753485227 ; Real: 13.0
prediccion: 15.68818615904344 ; Real: 26.6
prediccion: 28.03792474992337 ; Real: 26.0
prediccion: 14.554194048595914 ; Real: 16.5
prediccion: 30.76924317995298 ; Real: 20.0
prediccion: 27.867864362456274 ; Real: 24.5
prediccion: 30.026282831728736 ; Real: 32.2
prediccion: 26.479701606563616 ; Real: 28.0
prediccion: 23.907116347855574 ; Real: 20.0
prediccion: 30.612830475063664 ; Real: 26.0
prediccion: 14.163162286372629 ; Real: 18.1
prediccion: 20.986185942247708 ; Real: 18.0
prediccion: 30.436866182063188 ; Real: 37.7
prediccion: 20.7711184730249 ; Real: 21.5
prediccion: 28.370301747813162 ; Real: 21.6
prediccion: 30.475969358285514 ; Real: 31.3
prediccion: 14.124059110150302 

Observamos que el R2 es bajo y el ECM muy grande. 
Ejecutamos el modelo sin quitar ninguna caracteristica para observar sus resultados.

In [78]:
autos_df_ori_y = autos_df_ori.pop('mpg')

autos_df_ori_train, autos_df_ori_test,autos_df_ori_train_y, autos_df_ori_test_y = train_test_split(autos_df_ori, autos_df_ori_y)

modeloLineal.fit(autos_df_ori_train, autos_df_ori_train_y)
prediccion_ori = modeloLineal.predict(autos_df_ori_train)

print('R^2', modeloLineal.score(autos_df_ori_train, autos_df_ori_train_y))
print('Error cuadrático medio', mean_squared_error(prediccion_ori, autos_df_ori_train_y))
print('Error absoluto medio', mean_absolute_error(prediccion_ori, autos_df_ori_train_y))
print('Mediana del error absoluto', median_absolute_error(prediccion_ori, autos_df_ori_train_y))
print('-----------------------------------------------------------------------------')
prediccion_ori_test = modeloLineal.predict(autos_df_ori_test)
for index in range(len(prediccion_ori_test)):
    print('prediccion:', prediccion_ori_test[index], '; Real:', autos_df_ori_test_y.iloc[index])

R^2 0.8201990109414339
Error cuadrático medio 10.70426970290594
Error absoluto medio 2.484734193971636
Mediana del error absoluto 1.9983645799870686
-----------------------------------------------------------------------------
prediccion: 30.152642604935274 ; Real: 32.0
prediccion: 13.08561749188021 ; Real: 14.0
prediccion: 23.314767411691435 ; Real: 26.0
prediccion: 34.90724004115945 ; Real: 34.1
prediccion: 26.4233188121403 ; Real: 24.0
prediccion: 20.52661245457235 ; Real: 17.6
prediccion: 25.704649630654707 ; Real: 25.1
prediccion: 28.09178097102891 ; Real: 26.0
prediccion: 23.982283132472304 ; Real: 23.0
prediccion: 30.64685935110962 ; Real: 32.1
prediccion: 26.858134322302483 ; Real: 21.5
prediccion: 28.319405971573147 ; Real: 27.0
prediccion: 34.364711985577856 ; Real: 36.0
prediccion: 8.839802404107996 ; Real: 12.0
prediccion: 9.195569410555454 ; Real: 11.0
prediccion: 31.030287886104468 ; Real: 23.7
prediccion: 22.73184294136744 ; Real: 18.1
prediccion: 24.217873271245075 ; Re

Se observa gran mejoria en el R2 y en ECM.